<a href="https://colab.research.google.com/github/JayNguyen-123/Movie_Recommendation_System/blob/main/Movie_recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
!pip install surprise
from surprise import Reader, Dataset, SVD

import warnings; warnings.simplefilter('ignore')




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357288 sha256=69242c0687c7837e745de8be21cf4e89850fead9fb25087b512bb64ad60cef2a
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
# simple recommender

md = pd.read_csv('/content/movies_metadata.csv')
md.head(20)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0


In [3]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


In [4]:
#The next step is to determine an appropriate value for m, the minimum votes required to be listed in the chart.
# We will use 95th percentile as our cutoff.
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In [5]:
m = vote_counts.quantile(0.95)
m


434.0

In [6]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('_')[0] if x !=np.nan else np.nan)

qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title','year','vote_count','vote_average','popularity','genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape



(2274, 6)

In [7]:
"""to qualify to be considered for the chart, a movie has to have at least 434 votes on TMDB.
We also see that the average rating for a movie on TMDB is 5.244 on a scale of 10. 2274 Movies qualify to be on our chart.
Weighted Rating (WR) =  (vv+m.R)+(mv+m.C)

where,
v is the number of votes for the movie
m is the minimum votes required to be listed in the chart
R is the average rating of the movie
C is the mean vote across the whole report """

def weighted_rating(x):
  v = x['vote_count']
  R = x['vote_average']
  return (v/(v+m) * R) + (m/(m+v) * C)

qualified['wr'] = qualified.apply(weighted_rating, axis=1)
qualified = qualified.sort_values('wr', ascending=False).head(250)

qualified.head(15)


,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010-07-14 00:00:00,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008-07-16 00:00:00,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014-11-05 00:00:00,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999-10-15 00:00:00,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001-12-18 00:00:00,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994-09-10 00:00:00,8670,8,140.950236,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994-09-23 00:00:00,8358,8,51.645403,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003-12-01 00:00:00,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994-07-06 00:00:00,8147,8,48.307194,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002-12-18 00:00:00,7641,8,29.423537,"[Adventure, Fantasy, Action]",7.851924


In [8]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)


In [9]:
# we wil construct our function that builds charts for particular genres.
# For this, we will use relax our default conditions to the 85th percentile instead of 95.

def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)

    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')

    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)

    return qualified

build_chart('Romance').head(15)



,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995-10-20 00:00:00,661,9,34.457024,8.565285
351,Forrest Gump,1994-07-06 00:00:00,8147,8,48.307194,7.971357
876,Vertigo,1958-05-09 00:00:00,1162,8,18.20822,7.811667
40251,Your Name.,2016-08-26 00:00:00,1030,8,34.461252,7.789489
883,Some Like It Hot,1959-03-18 00:00:00,835,8,11.845107,7.745154
1132,Cinema Paradiso,1988-11-17 00:00:00,834,8,14.177005,7.744878
19901,Paperman,2012-11-02 00:00:00,734,8,7.198633,7.713951
37863,Sing Street,2016-03-11 00:00:00,669,8,10.672862,7.689483
882,The Apartment,1960-06-15 00:00:00,498,8,11.994281,7.599317
38718,The Handmaiden,2016-06-01 00:00:00,453,8,16.727405,7.566166


In [10]:

# personalise our recommendations more, we are going to build an engine that computes similarity between movies based on certain metrics and suggests
# movies that are most similar to a particular movie that a user liked.
# we will be using movie metadata (or content) to build this engine, this also known as Content Based Filtering.

links_small = pd.read_csv('/content/links_small.csv')
links_small.head()



,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [11]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')



In [12]:
md = md.drop([19730, 29503, 35587])


In [13]:
md['id'] = md['id'].astype('int')


In [14]:
smd = md[md['id'].isin(links_small)]
smd.shape


(9099, 25)

In [15]:
# Movie Description Based Recommender
# Let us first try to build a recommender using movie descriptions and taglines.
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=0.0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

tfidf_matrix.shape



(9099, 268124)

In [16]:
# Cosine Similarity
# Mathematically, it is defined as follows:
# cosine(x,y)=x.y⊺||x||.||y||
#  we will use sklearn's linear_kernel instead of cosine_similarities since it is much faster

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[0]



array([1.        , 0.00680476, 0.        , ..., 0.        , 0.00344913,
       0.        ])

In [17]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])


In [18]:
def get_recommendations(title):
  idx = indices[title]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:31]
  movie_indices = [i[0] for i in sim_scores]

  return titles.iloc[movie_indices]

get_recommendations('The Godfather').head(10)



,title
973,The Godfather: Part II
8387,The Family
3509,Made
4196,Johnny Dangerously
29,Shanghai Triad
5667,Fury
2412,American Movie
1582,The Godfather: Part III
4221,8 Women
2159,Summer of Sam


In [19]:
get_recommendations('The Dark Knight').head(10)


,title
7931,The Dark Knight Rises
132,Batman Forever
1113,Batman Returns
8227,"Batman: The Dark Knight Returns, Part 2"
7565,Batman: Under the Red Hood
524,Batman
7901,Batman: Year One
2579,Batman: Mask of the Phantasm
2696,JFK
8165,"Batman: The Dark Knight Returns, Part 1"


In [20]:
# Metadata Based Recommender
# To build our standard metadata based content recommender, we will need to merge our current dataset with the crew and the keyword datasets.
# Let us prepare this data as our first step.

credits = pd.read_csv('/content/credits.csv')
keywords = pd.read_csv('/content/keywords.csv')

keywords['id'] = keywords['id'].astype('int')
credits['ide'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

md.shape


(45463, 25)

In [21]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 29)

In [22]:
""" We now have our cast, crew, genres and credits, all in one dataframe. Let us wrangle this a little more using the following intuitions:

Crew: From the crew, we will only pick the director as our feature since the others don't contribute that much to the feel of the movie.
Cast: Choosing Cast is a little more tricky. Lesser known actors and minor roles do not really affect people's opinion of a movie.
Therefore, we must only select the major characters and their respective actors. Arbitrarily we will choose the top 3 actors that appear in the credits list."""

smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)

smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))


In [23]:
def get_director(x):
  for i in x:
    if i['job'] == 'Director':
      return i['name']
    return np.nan

smd['director'] = smd['crew'].apply(get_director)
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >= 3 else x)

smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


In [24]:
""" These are steps I follow in the preparation of my genres and credits data:
Strip Spaces and Convert to Lowercase from all our features. This way, our engine will not confuse between Johnny Depp and Johnny Galecki.
Mention Director 3 times to give it more weight relative to the entire cast."""
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x,x])


In [25]:
""" Keywords:
We will do a small amount of pre-processing of our keywords before putting them to any use. As a first step,
we calculate the frequenct counts of every keyword that appears in the dataset."""

s = smd.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

s = s.value_counts()
s[:5]


,count
keyword,
independent film,610
woman director,550
murder,399
duringcreditsstinger,327
based on novel,318


In [26]:
s = s[s > 1]

In [27]:
#  we will convert every word to its stem so that words such as Dogs and Dog are considered the same.

stemmer = SnowballStemmer('english')
stemmer.stem('dogs')


'dog'

In [28]:
def filter_keywords(x):
  words = []
  for i in x:
    if i in s:
      words.append(i)
  return words

smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ","")) for i in x])

smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

count = CountVectorizer(analyzer='word', ngram_range=(1,2), min_df=0.0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

cosine_sim = cosine_similarity(count_matrix, count_matrix)

smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

get_recommendations('The Dark Knight').head(10)




,title
7991,The Dark Knight Rises
5070,The Enforcer
6186,Batman Begins
5907,Thursday
217,The Glass Shield
2264,In Too Deep
6317,Menolippu Mombasaan
7986,Bullet to the Head
7619,Batman: Under the Red Hood
5045,The Statement


In [29]:
get_recommendations('Mean Girls').head(10)


,title
8033,Dylan Moran: What It Is
5186,Swimming to Cambodia
7386,Werner Herzog Eats His Shoe
7426,The Fly
8173,Lola Versus
418,House Party 3
2663,Class Reunion
2773,National Lampoon's Last Resort
3084,Easy Money
3416,Nine to Five


In [31]:
""" Although Batman and Robin has a lot of similar characters as compared to The Dark Knight but it was a terrible movie that shouldn't be recommended to anyone.
Therefore, we will add a mechanism to remove bad movies and return movies which are popular and have had a good critical response."""

def improved_recommendations(title):
  idx= indices[title]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:26]
  movie_indices = [i[0] for i in sim_scores]

  movies = smd.iloc[movie_indices][['title','vote_count','vote_average','year']]
  vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
  vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
  C = vote_averages.mean()
  m = vote_counts.quantile(0.60)

  qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & movies['vote_average'].notnull()]
  qualified['vote_count'] = qualified['vote_count'].astype('int')
  qualified['vote_average'] = qualified['vote_average'].astype('int')
  qualified['wr'] = qualified.apply(weighted_rating, axis=1)
  qualified = qualified.sort_values('wr', ascending=False).head(10)

  return qualified

improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7991,The Dark Knight Rises,9263,7,2012-07-16 00:00:00,6.921448
6186,Batman Begins,7511,7,2005-06-10 00:00:00,6.904127
7619,Batman: Under the Red Hood,459,7,2010-07-27 00:00:00,6.147016
7336,Bronson,756,6,2008-10-09 00:00:00,5.724609
7872,Takers,399,6,2010-08-26 00:00:00,5.606585
6519,Chaos,278,6,2005-01-17 00:00:00,5.539726
7633,Animal Kingdom,240,6,2010-06-03 00:00:00,5.513776
4843,Sleeping with the Enemy,228,6,1991-02-08 00:00:00,5.504962
5619,Ladder 49,213,6,2004-10-01 00:00:00,5.493486
7986,Bullet to the Head,490,5,2013-01-31 00:00:00,5.115027


In [32]:
improved_recommendations('Mean Girls')


,title,vote_count,vote_average,year,wr
7922,Chinese Take-Out,95,7,2011-03-24 00:00:00,5.560085
8415,Louis C.K.: Oh My God,66,7,2013-04-13 00:00:00,5.476570
8858,Louis C.K.: Live at The Comedy Store,58,7,2015-01-27 00:00:00,5.451799
3416,Nine to Five,102,6,1980-12-18 00:00:00,5.388592
8675,Premature,120,5,2014-07-02 00:00:00,5.191850
8276,Parental Guidance,212,5,2012-12-25 00:00:00,5.164528
8662,Tammy,508,5,2014-07-02 00:00:00,5.112829
5112,Welcome to Mooseport,54,4,2004-02-24 00:00:00,5.107142
8173,Lola Versus,55,4,2012-04-24 00:00:00,5.104878
8087,The Dictator,1789,5,2012-05-15 00:00:00,5.047812


In [33]:
# Collaborative Filtering
#Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will like a particular product or
# service those users have used/experienced but I have not.
#  We will use the Surprise library that used extremely powerful algorithms like Singular Value Decomposition (SVD) to minimise RMSE (Root Mean Square Error)
# and give great recommendations.

reader = Reader()
ratings = pd.read_csv('/content/ratings_small.csv')

ratings.head()


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [34]:
from surprise.model_selection import cross_validate

data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)


svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8877  0.8986  0.9033  0.8989  0.8915  0.8960  0.0056  
MAE (testset)     0.6820  0.6935  0.6954  0.6913  0.6870  0.6899  0.0048  
Fit time          2.26    2.33    1.61    1.58    1.57    1.87    0.35    
Test time         0.23    0.55    0.18    0.15    0.12    0.25    0.16    


{'test_rmse': array([0.88768707, 0.89856998, 0.90331543, 0.89888353, 0.89149938]),
 'test_mae': array([0.68200634, 0.69352124, 0.69542109, 0.69127919, 0.68702878]),
 'fit_time': (2.2556159496307373,
  2.3305416107177734,
  1.605952262878418,
  1.5788581371307373,
  1.5747950077056885),
 'test_time': (0.22508549690246582,
  0.5530092716217041,
  0.17876696586608887,
  0.15288901329040527,
  0.1210336685180664)}

In [36]:
from surprise import Dataset, KNNBasic
trainset = data.build_full_trainset()
algo = KNNBasic()
algo.fit(trainset)




Computing the msd similarity matrix...
Done computing similarity matrix.


In [37]:
ratings[ratings['userId'] == 1]


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [38]:
algo.predict(1, 302, 3)


Prediction(uid=1, iid=302, r_ui=3, est=3.6890085183191363, details={'actual_k': 5, 'was_impossible': False})

In [41]:
# Hybrid Recommender
# we will build a simple hybrid recommender that brings together techniques we have implemented in the content based and collaborative filter based engines.

def convert_int(x):
  try:
    return int(x)
  except:
    return np.nan

id_map = pd.read_csv('/content/links_small.csv')[['movieId','tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId','id']
id_map = id_map.merge(smd[['title','id']], on='id').set_index('title')

indices_map = id_map.set_index('id')

def hybrid_recommendation(userId, title):
  idx = indices[title]
  tmdbId = id_map.loc[title]['movieId']

  sim_scores = list(enumerate(cosine_sim[int(idx)]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:26]
  movie_indices = [i[0] for i in sim_scores]

  movies = smd.iloc[movie_indices][['title','vote_count','vote_average','year','id']]
  movies['est'] = movies['id'].apply(lambda x: algo.predict(userId, indices_map.loc[x]['movieId']).est)
  movies = movies.sort_values('est', ascending=False)

  return movies.head(10)

hybrid_recommendation(1,'Avatar')




,title,vote_count,vote_average,year,id,est
7669,Alice in Wonderland,8.0,5.4,1933-12-22 00:00:00,25694,4.500000
8357,Star Trek Into Darkness,4479.0,7.4,2013-05-05 00:00:00,54138,4.140395
7426,The Fly,10.0,7.4,1980-09-05 00:00:00,47466,3.543608
7348,The Three Musketeers,2.0,3.0,1933-04-07 00:00:00,140887,3.543608
8515,Zatôichi on the Road,11.0,7.5,1963-11-30 00:00:00,18624,3.543608
3919,Impostor,136.0,6.1,2001-12-03 00:00:00,4965,3.500000
4001,Hawk the Slayer,13.0,4.5,1980-08-27 00:00:00,25628,3.500000
7172,Replicant,93.0,5.0,2001-05-11 00:00:00,10596,3.500000
3052,Sinbad and the Eye of the Tiger,39.0,6.3,1977-07-15 00:00:00,11940,3.434212
8975,The Hunger Games: Mockingjay - Part 2,4117.0,6.6,2015-11-18 00:00:00,131634,3.378981


In [42]:
hybrid_recommendation(500, 'Avatar')


,title,vote_count,vote_average,year,id,est
7669,Alice in Wonderland,8.0,5.4,1933-12-22 00:00:00,25694,4.500000
8515,Zatôichi on the Road,11.0,7.5,1963-11-30 00:00:00,18624,4.500000
7348,The Three Musketeers,2.0,3.0,1933-04-07 00:00:00,140887,4.000000
8357,Star Trek Into Darkness,4479.0,7.4,2013-05-05 00:00:00,54138,3.903589
7172,Replicant,93.0,5.0,2001-05-11 00:00:00,10596,3.500000
4001,Hawk the Slayer,13.0,4.5,1980-08-27 00:00:00,25628,3.500000
3919,Impostor,136.0,6.1,2001-12-03 00:00:00,4965,3.500000
3052,Sinbad and the Eye of the Tiger,39.0,6.3,1977-07-15 00:00:00,11940,3.284674
3878,Spacehunter: Adventures in the Forbidden Zone,37.0,5.1,1983-05-20 00:00:00,26978,3.000000
8986,Star Trek Beyond,2636.0,6.6,2016-07-07 00:00:00,188927,2.723251


In [ ]:
""" We see that for our hybrid recommender, we get different recommendations for different users although the movie is the same.
The recommendation is more personalized and tailored towards particular users."""

